In [1]:
!git clone https://huggingface.co/spaces/samim2024/PDF-RAG

Cloning into 'PDF-RAG'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 63 (delta 17), reused 0 (delta 0), pack-reused 28 (from 1)
Unpacking objects: 100% (63/63), 23.96 KiB | 1.04 MiB/s, done.


In [1]:
!huggingface-cli login





    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read)

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(model="mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16, device_map="auto")
output = pipe("how are you?", do_sample=True, top_p=0.95)

print(output)

In [ ]:
!pip install -r /content/PDF-RAG/requirements.txt

In [3]:
!pip install pyngrok

In [4]:
!ngrok config add-authtoken XXXXXXXXXXXXXXXXXXXXXXXXXXXX

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:




from pyngrok import ngrok
import os

# بدء نفق ngrok على المنفذ 8501
port = 8501
public_url = ngrok.connect(port)

print(f"ngrok tunnel is running. Public URL: {public_url}")

# تشغيل تطبيق Streamlit
os.system(f"streamlit run /content/PDF-RAG/app.py --server.port=8501 &")

ngrok tunnel is running. Public URL: NgrokTunnel: "https://e7e5-35-194-151-180.ngrok-free.app" -> "http://localhost:8501"


0

In [ ]:


from pyngrok import ngrok
import os

# بدء نفق ngrok على المنفذ 8501
port = 8501
public_url = ngrok.connect(port)

print(f"ngrok tunnel is running. Public URL: {public_url}")










from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
#from langchain_community.llms import Ollama
#from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import streamlit as st
import os
import time
from langchain_community.llms import HuggingFaceEndpoint
import torch

from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists('jj'):
    os.mkdir('jj')

if 'template' not in st.session_state:
    st.session_state.template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""
if 'prompt' not in st.session_state:
    st.session_state.prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=st.session_state.template,
    )
if 'memory' not in st.session_state:
    st.session_state.memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question")
if 'vectorstore' not in st.session_state:
    #st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=OllamaEmbeddings(base_url='http://localhost:11434',model="mistral")
    st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=embeddings)

if 'llm' not in st.session_state:
    #st.session_state.llm = Ollama(base_url="http://localhost:11434",model="mistral",verbose=True,callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),)
    st.session_state.llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float32, device_map="auto", Temperature=0.9)

# Initialize session state
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

st.title("PDF Chatbot")

# Upload a PDF file
uploaded_file = st.file_uploader("Upload your PDF", type='pdf')

for message in st.session_state.chat_history:
    with st.chat_message(message["role"]):
        st.markdown(message["message"])

if uploaded_file is not None:
    if not os.path.isfile("files/"+uploaded_file.name+".pdf"):
        with st.status("Analyzing your document..."):
            bytes_data = uploaded_file.read()
            f = open("files/"+uploaded_file.name+".pdf", "wb")
            f.write(bytes_data)
            f.close()
            loader = PyPDFLoader("files/"+uploaded_file.name+".pdf")
            data = loader.load()

            # Initialize text splitter
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1500,
                chunk_overlap=0,
                length_function=len
            )
            all_splits = text_splitter.split_documents(data)

            # Create and persist the vector store
            #st.session_state.vectorstore = Chroma.from_documents(documents=all_splits,embedding=OllamaEmbeddings(model="mistral"))
            st.session_state.vectorstore = Chroma.from_documents(documents=all_splits,embedding=embeddings)
            st.session_state.vectorstore.persist()

    st.session_state.retriever = st.session_state.vectorstore.as_retriever()
    # Initialize the QA chain
    if 'qa_chain' not in st.session_state:
        st.session_state.qa_chain = RetrievalQA.from_chain_type(
            llm=st.session_state.llm,
            chain_type='stuff',
            retriever=st.session_state.retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": st.session_state.prompt,
                "memory": st.session_state.memory,
            }
        )

    # Chat input
    if user_input := st.chat_input("You:", key="user_input"):
        user_message = {"role": "user", "message": user_input}
        st.session_state.chat_history.append(user_message)
        with st.chat_message("user"):
            st.markdown(user_input)
        with st.chat_message("assistant"):
            with st.spinner("Assistant is typing..."):
                response = st.session_state.qa_chain(user_input)
            message_placeholder = st.empty()
            full_response = ""
            for chunk in response['result'].split():
                full_response += chunk + " "
                time.sleep(0.05)
                # Add a blinking cursor to simulate typing
                message_placeholder.markdown(full_response + "▌")
            message_placeholder.markdown(full_response)

        chatbot_message = {"role": "assistant", "message": response['result']}
        st.session_state.chat_history.append(chatbot_message)


else:
    st.write("Please upload a PDF file.")

In [4]:
import streamlit as st
import os
import time
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.manager import CallbackManager
#from langchain_community.llms import Ollama
#from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
#from langchain_community.llms import HuggingFaceEndpoint #Fixed: Import was not being used
import torch

from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline #Fixed: Added import for pipeline

# Initialize pipeline outside of session state, otherwise it will re-initialize every time
pipe = pipeline(model="mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16, device_map="auto")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists('jj'):
    os.mkdir('jj')


#Fixed: Initialize session state variables if they don't exist
if 'template' not in st.session_state:
    st.session_state.template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""
if 'prompt' not in st.session_state:
    st.session_state.prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=st.session_state.template, # This now refers to an existing key
    )
if 'memory' not in st.session_state:
    st.session_state.memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question")
if 'vectorstore' not in st.session_state:
    #st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=OllamaEmbeddings(base_url='http://localhost:11434',model="mistral")
    st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=embeddings)

if 'llm' not in st.session_state:
    #st.session_state.llm = Ollama(base_url="http://localhost:11434",model="mistral",verbose=True,callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),)
    st.session_state.llm = pipe #Fixed: Use the initialized pipeline instead of HuggingFaceEndpoint


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists('jj'):
    os.mkdir('jj')

if 'template' not in st.session_state:
    st.session_state.template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative.Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""
if 'prompt' not in st.session_state:
    st.session_state.prompt = PromptTemplate(
        input_variables=["history", "context", "question"],
        template=st.session_state.template,
    )
if 'memory' not in st.session_state:
    st.session_state.memory = ConversationBufferMemory(
        memory_key="history",
        return_messages=True,
        input_key="question")
if 'vectorstore' not in st.session_state:
    #st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=OllamaEmbeddings(base_url='http://localhost:11434',model="mistral")
    st.session_state.vectorstore = Chroma(persist_directory='jj', embedding_function=embeddings)

if 'llm' not in st.session_state:
    #st.session_state.llm = Ollama(base_url="http://localhost:11434",model="mistral",verbose=True,callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),)
    st.session_state.llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float32, device_map="auto", Temperature=0.9)

# Initialize session state
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

st.title("PDF Chatbot")

# Upload a PDF file
uploaded_file = st.file_uploader("Upload your PDF", type='pdf')

for message in st.session_state.chat_history:
    with st.chat_message(message["role"]):
        st.markdown(message["message"])

if uploaded_file is not None:
    if not os.path.isfile("files/"+uploaded_file.name+".pdf"):
        with st.status("Analyzing your document..."):
            bytes_data = uploaded_file.read()
            f = open("files/"+uploaded_file.name+".pdf", "wb")
            f.write(bytes_data)
            f.close()
            loader = PyPDFLoader("files/"+uploaded_file.name+".pdf")
            data = loader.load()

            # Initialize text splitter
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1500,
                chunk_overlap=0,
                length_function=len
            )
            all_splits = text_splitter.split_documents(data)

            # Create and persist the vector store
            #st.session_state.vectorstore = Chroma.from_documents(documents=all_splits,embedding=OllamaEmbeddings(model="mistral"))
            st.session_state.vectorstore = Chroma.from_documents(documents=all_splits,embedding=embeddings)
            st.session_state.vectorstore.persist()

    st.session_state.retriever = st.session_state.vectorstore.as_retriever()
    # Initialize the QA chain
    if 'qa_chain' not in st.session_state:
        st.session_state.qa_chain = RetrievalQA.from_chain_type(
            llm=st.session_state.llm,
            chain_type='stuff',
            retriever=st.session_state.retriever,
            verbose=True,
            chain_type_kwargs={
                "verbose": True,
                "prompt": st.session_state.prompt,
                "memory": st.session_state.memory,
            }
        )

    # Chat input
    if user_input := st.chat_input("You:", key="user_input"):
        user_message = {"role": "user", "message": user_input}
        st.session_state.chat_history.append(user_message)
        with st.chat_message("user"):
            st.markdown(user_input)
        with st.chat_message("assistant"):
            with st.spinner("Assistant is typing..."):
                response = st.session_state.qa_chain(user_input)
            message_placeholder = st.empty()
            full_response = ""
            for chunk in response['result'].split():
                full_response += chunk + " "
                time.sleep(0.05)
                # Add a blinking cursor to simulate typing
                message_placeholder.markdown(full_response + "▌")
            message_placeholder.markdown(full_response)

        chatbot_message = {"role": "assistant", "message": response['result']}
        st.session_state.chat_history.append(chatbot_message)


else:
    st.write("Please upload a PDF file.")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


AttributeError: st.session_state has no attribute "template". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization

In [5]:
from huggingface_hub import login
login("your_huggingface_api_token")


HTTPError: Invalid user token. If you didn't pass a user token, make sure you are properly logged in by executing `huggingface-cli login`, and if you did pass a user token, double-check it's correct.

In [ ]:
from huggingface_hub import whoami
print(whoami())


ngrok tunnel is running. Public URL: NgrokTunnel: "https://ce57-35-194-151-180.ngrok-free.app" -> "http://localhost:8501"


0

In [2]:
!export HUGGINGFACE_HUB_TOKEN ""

In [4]:
!ping huggingface.co

/bin/bash: line 1: ping: command not found


In [ ]:
from huggingface_hub import whoami
print(whoami())


In [6]:
import os
from huggingface_hub import login

# Set token in environment variable
os.environ["HUGGINGFACE_HUB_TOKEN"] = ""

# Authenticate
login()


In [8]:
import os

# Replace with your Hugging Face API token
os.environ["HF_TOKEN"] = ""

from huggingface_hub import login

# Log in using the token
login(token=os.environ["HF_TOKEN"])


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [14]:
from langchain_community.llms import HuggingFaceEndpoint

st.session_state.llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    torch_dtype="float32",
    device_map="auto",
    Temperature=0.9
)


                    torch_dtype was transferred to model_kwargs.
                    Please make sure that torch_dtype is what you intended.
                    device_map was transferred to model_kwargs.
                    Please make sure that device_map is what you intended.
                    Temperature was transferred to model_kwargs.
                    Please make sure that Temperature is what you intended.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


NameError: name 'st' is not defined

In [ ]:
from huggingface_hub import whoami

print(whoami())



In [3]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"


In [ ]:
from huggingface_hub import whoami
print(whoami())


In [3]:
from langchain_community.llms import HuggingFaceEndpoint
import os
st.session_state.llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    temperature=0.7,
)


ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Could not authenticate with huggingface_hub. Please check your API token. (type=value_error)

In [4]:
!pip install huggingface_hub

In [2]:
import os
from langchain_community.llms import HuggingFaceEndpoint
import streamlit as st

# Replace 'YOUR_HUGGINGFACE_API_TOKEN' with your actual token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HUGGINGFACE_API_TOKEN"

# Or, if you've already logged in using the huggingface-cli:
# from huggingface_hub import hf_hub_download
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_hub_download(repo_id="gpt2", filename="config.json", use_auth_token=True)

st.session_state.llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"], # Pass the token explicitly
    temperature=0.7,
)

ValidationError: 1 validation error for HuggingFaceEndpoint
__root__
  Could not authenticate with huggingface_hub. Please check your API token. (type=value_error)

In [ ]:
import os
from langchain_community.llms import HuggingFaceEndpoint
import streamlit as st

# 1. Replace 'YOUR_HUGGINGFACE_API_TOKEN' with your actual token from Hugging Face
YOUR_HUGGINGFACE_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"
os.environ["HUGGINGFACEHUB_API_TOKEN"] = YOUR_HUGGINGFACE_API_TOKEN

# 2. Pass the token explicitly when creating HuggingFaceEndpoint
st.session_state.llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    huggingfacehub_api_token=YOUR_HUGGINGFACE_API_TOKEN, # Pass the token directly
    temperature=0.7,
)


لتسريع المعالجة ، تحتاج إلى تحديد device_map="auto" في HuggingFaceEndpoint أو HuggingFacePipeline.
تأكد من تثبيت مكتبة accelerate

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    temperature=0.7,
)

In [4]:


!export HUGGINGFACE_HUB_TOKEN "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"

from pyngrok import ngrok
import os

# بدء نفق ngrok على المنفذ 8501
port = 8501
public_url = ngrok.connect(port)

print(f"ngrok tunnel is running. Public URL: {public_url}")

# تشغيل تطبيق Streamlit
os.system(f"streamlit run /content/PDF-RAG/app.py --server.port=8501 &")

ngrok tunnel is running. Public URL: NgrokTunnel: "https://be6e-35-194-151-180.ngrok-free.app" -> "http://localhost:8501"


0

In [6]:


!export HUGGINGFACE_HUB_TOKEN "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"

from pyngrok import ngrok
import os

# بدء نفق ngrok على المنفذ 8501
port = 8501
public_url = ngrok.connect(port)

print(f"ngrok tunnel is running. Public URL: {public_url}")

# تشغيل تطبيق Streamlit
os.system(f"streamlit run /content/PDF-RAG/a.py --server.port=8501 &")

ngrok tunnel is running. Public URL: NgrokTunnel: "https://75af-35-194-151-180.ngrok-free.app" -> "http://localhost:8501"


0

In [8]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/PDF-RAG/temp.pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: II diabetes mellitus, hypertension, elevated lipids, gastroesophageal reflux disease (GERD), diverticulosis, endometriosis, kidney stones, anxiety and osteopenia. Her medications included atorvastatin, Inderal, lisinopril, metformin, omeprazole and Paxil. She had allergies to dyazide, morphine, intravenous contrast dye, aspirin and triamcinolone cream. She was a retired research assistant and was married with two children. She never smoked and did not drink alcohol. There was no family history of stomach cancer, but her older brother died of colon cancer metastatic to the liver and lung at the age of 90.
On examination, the weight was 182 pounds, the 
blood pressure

In [1]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/PDF-RAG/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no long



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: I could start at any point in my short miserable life 
to prove it, but things really started going bad last May, 
when our sixth-grade class took a field trip to Manhattan— 
twenty-eight mental-case kids and two teachers on a yellow 
school bus, heading to the Metropolitan Museum of Art to 
look at ancient Greek and Roman stuff. 
I know—it sounds like torture. Most Yancy field trips 
were. 
But Mr. Brunner, our Latin teacher, was leading this 
trip, so I had hopes. 
Mr. Brunner was this middle-aged guy in a motorized 
wheelchair. He had thinning hair and a scruffy beard and a 
frayed tweed jacket, which always smelled like coffee. You 
wouldn't think he'd be cool, 

In [1]:
from langchain.chains import RetrievalQA
from langchain.callbacks.manager import CallbackManager
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.embeddings import HuggingFaceEmbeddings

# --- Configuration ---
HUGGINGFACEHUB_API_TOKEN = "XXXXXXXXXXXXXXXXXXXXXXXXXXXX"  # Replace with your actual token
PDF_FILE_PATH = "/content/PDF-RAG/Clas Blomberg - Physics of life-Elsevier Science (2007).pdf"  # Replace with the path to your PDF file
QUESTION = "What is the main topic of this document?"  # Replace with your question
PERSIST_DIRECTORY = "jj"
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
MODEL_KWARGS = {'device': 'cpu'}
ENCODE_KWARGS = {'normalize_embeddings': False}
REPO_ID = "mistralai/Mistral-7B-Instruct-v0.2"
TEMPERATURE = 0.9

# --- Setup ---
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

# Create directories if they don't exist
if not os.path.exists('files'):
    os.mkdir('files')

if not os.path.exists(PERSIST_DIRECTORY):
    os.mkdir(PERSIST_DIRECTORY)

# --- Embeddings ---
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs=MODEL_KWARGS,
    encode_kwargs=ENCODE_KWARGS
)

# --- Prompt Template ---
template = """You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: {context}
    History: {history}

    User: {question}
    Chatbot:"""

prompt = PromptTemplate(
    input_variables=["history", "context", "question"],
    template=template,
)

# --- Memory ---
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
    input_key="question"
)

# --- Vector Store ---
if not os.path.exists(os.path.join(PERSIST_DIRECTORY, "index")):
    # Load and process the PDF only if the index doesn't exist
    loader = PyPDFLoader(PDF_FILE_PATH)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=0,
        length_function=len
    )
    all_splits = text_splitter.split_documents(data)

    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIRECTORY
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings
    )

# --- Language Model ---
llm = HuggingFaceEndpoint(
    repo_id=REPO_ID,
    temperature=TEMPERATURE,
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

# --- Retriever ---
retriever = vectorstore.as_retriever()

# --- Retrieval QA Chain ---
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=retriever,
    verbose=True,
    chain_type_kwargs={
        "verbose": True,
        "prompt": prompt,
        "memory": memory,
    }
)

# --- Run the QA Chain ---
response = qa_chain(QUESTION)

# --- Print the Result ---
print(f"Question: {QUESTION}")
print(f"Answer: {response['result']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no long



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are a knowledgeable chatbot, here to help with questions of the user. Your tone should be professional and informative. Try to give answer in tabular and shortcut.

    Context: Printed and bound in The Netherlands
0 70 80 91 01 1 1 0987654321
For information on all Elsevier publications
visit our website at books.elsevier.com

correlation, 136, 167–168, 181, 208–209,
218–219, 235, 238, 253, 300, 304–305, 307,
313–316, 319, 360
cosmological, 2, 404cost (of selection), 323, 325, 327–328,
335–336, 338, 340, 357, 361, 377
cost of information, 340
Coulomb’s law, 26, 29–31critical phenomena, 154, 166cyclic AMP, 272, 278cytosine, 101, 106, 355, 359–360
D-amino acid, 111, 355
death, 5, 50, 116, 201, 203, 340, 403–406degrees of freedom, 11, 21, 45, 87, 121, 217,
221, 252, 299, 386
deoxyribose, 105, 107, 111determinism, 2, 4–5, 62, 289, 300, 383–385